In [14]:
%load_ext autoreload
%autoreload 2

In [1]:
!pip install transformer_lens -q
!pip install --upgrade attrs -q
!pip uninstall jax jaxlib -y -q
!pip install jax jaxlib -q

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
scipy 1.9.2 requires numpy<1.26.0,>=1.18.5, but you have numpy 1.26.4 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gradient 2.0.6 requires attrs<=19, but you have attrs 23.2.0 which is incompatible.


In [ ]:
from transformer_lens import HookedTransformer
import torch

import json
import time

from src.neuron_texts import find_truncated_texts

In [13]:
# Load jsons from ../experiment_data/1_next_token_neurons
# filename = "2023-10-14_17-23-35_gpt2-large_mid_test"
# filename = "2024-02-13_06-56-03_pythia-160m"
# filename = "2024-02-13_07-47-52_pythia-410m"
# filename = "2024-02-13_09-18-06_pythia-1.4b"

filename = "2024-02-13_11-37-07_pythia-1.4b_test"
filename = "2024-02-13_12-53-32_pythia-410m_test"


train = False

with open(f'./experiment_data/2_max_activating_texts/{filename}.json') as f:
    max_text_data = json.load(f)

In [10]:
!ls experiment_data/2_max_activating_texts/

/usr/lib/python3.9/pty.py:85: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


2023-10-11_14-06-30_gpt2-small.json
2023-10-11_14-29-40_gpt2-medium.json
2023-10-11_14-30-39_gpt2-medium.json
2023-10-11_16-36-33_gpt2-large.json
2023-10-13_03-52-29_gpt2-small_test.json
2023-10-13_04-29-47_gpt2-medium_test.json
2023-10-13_10-22-07_gpt2-large_test.json
2023-10-13_15-31-49_gpt2-large_mid.json
2023-10-13_19-47-54_gpt2-large_mid.json
2023-10-14_17-23-35_gpt2-large_mid_test.json
2024-02-13_06-56-03_pythia-160m.json
2024-02-13_07-47-52_pythia-410m.json
2024-02-13_09-18-06_pythia-1.4b.json
2024-02-13_11-37-07_pythia-1.4b_test.json
2024-02-13_12-53-32_pythia-410m_test.json
2024-02-13_13-17-59_pythia-160m_test.json


In [ ]:
# Parameters
parameters = max_text_data['parameters']
model_name = parameters['model_name']

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
model = HookedTransformer.from_pretrained(
    model_name,
    center_unembed=True,
    center_writing_weights=True,
    fold_ln=True,
    # refactor_factored_attn_matrices=True,
    device=device,
)

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.93G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/396 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loaded pretrained model pythia-1.4b into HookedTransformer


In [11]:
if train:
    from datasets import load_dataset
    dataset = load_dataset("NeelNanda/pile-10k", split="train")
    dataset_text_list = [x['text'] for x in dataset]
    print("Train loaded!")

else:

    import pickle

    with open('./text_list_dict_test.pkl', 'rb') as f:
        dataset_text_dict = pickle.load(f)

    dataset_text_list = [x['text'] for x in dataset_text_dict]
    dataset = dataset_text_dict

    print("test loaded!")

test loaded!


In [12]:
with torch.no_grad():
    neuron_20_examples = find_truncated_texts(
        model=model,
        neuron_max_acts=max_text_data['neuron_max_acts'],
        dataset=dataset,
        device=device,
        num_samples=30,
        act_ratio=0.8,
    )

output = {
    'parameters': parameters,
    'neuron_to_trunc_data': neuron_20_examples,
    'prior_filename': filename,
}


train_string = "train" if train else "test_fr"
# Save json to ../experiment_data/2_max_activating_texts
timestamp = time.strftime("%Y-%m-%d_%H-%M-%S", time.localtime(int(time.time())))
new_filename = f"{timestamp}_{model_name}_{train_string}.json"

with open(f'./experiment_data/3_trimmed_texts/{new_filename}', 'w') as f:
    json.dump(output, f)

30it [00:25,  1.18it/s]00:00<?, ?it/s]
30it [00:23,  1.25it/s]00:25<33:26, 25.40s/it]
30it [00:28,  1.05it/s]00:49<31:57, 24.59s/it]
30it [00:30,  1.02s/it]01:17<33:51, 26.38s/it]
30it [00:30,  1.01s/it]01:48<35:30, 28.04s/it]
30it [00:30,  1.03s/it]02:18<36:06, 28.89s/it]
30it [00:22,  1.36it/s]02:49<36:28, 29.58s/it]
30it [00:30,  1.03s/it]03:11<33:01, 27.14s/it]
30it [00:23,  1.30it/s]03:42<34:00, 28.34s/it]
30it [00:27,  1.10it/s]04:06<31:37, 26.72s/it]
30it [00:30,  1.00s/it][04:33<31:21, 26.88s/it]
30it [00:31,  1.06s/it][05:03<32:03, 27.88s/it]
30it [00:26,  1.15it/s][05:35<32:56, 29.06s/it]
30it [00:25,  1.17it/s][06:01<31:25, 28.15s/it]
30it [00:26,  1.12it/s][06:26<30:07, 27.39s/it]
30it [00:23,  1.25it/s][06:53<29:27, 27.20s/it]
30it [00:30,  1.01s/it][07:17<27:57, 26.21s/it]
30it [00:29,  1.02it/s][07:47<28:47, 27.42s/it]
30it [00:30,  1.01s/it][08:17<28:56, 28.01s/it]
30it [00:24,  1.20it/s][08:47<29:09, 28.67s/it]
30it [00:27,  1.10it/s][09:12<27:33, 27.56s/it]
30it [00:2

In [7]:
output = {
    'parameters': parameters,
    'neuron_to_trunc_data': neuron_20_examples,
    'prior_filename': filename,
}


train_string = "train" if train else "test"
# Save json to ../experiment_data/2_max_activating_texts
timestamp = time.strftime("%Y-%m-%d_%H-%M-%S", time.localtime(int(time.time())))
new_filename = f"{timestamp}_{model_name}_{train_string}.json"

with open(f'./experiment_data/3_trimmed_texts/{new_filename}', 'w') as f:
    json.dump(output, f)